In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


Workspace name: quick-starts-ws-130459
Azure region: southcentralus
Subscription id: 572f8abf-a1a0-4b78-8c6d-3630739c72b5
Resource group: aml-quickstarts-130459


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

#name for cpu cluster
amlcompute_cluster_name = "cpu-cluster"

#check for existance of cluster
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, using it ...')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes = 4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
aml_compute.wait_for_completion(show_output=True)


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
    "--C": uniform(0.001,1),
    "--max_iter": choice([100, 200, 300, 500])
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.01, evaluation_interval=1, delay_evaluation=3)
import shutil
if "training" not in os.listdir():
    os.mkdir("./training")
os.makedirs("./training", exist_ok=True)
shutil.copy("train.py","./training")
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './training',
              compute_target=aml_compute,
              entry_script='train.py',
              pip_packages=["pyarrow>=0.12.0", "pyspark"])
primary_metric_name = "Accuracy"
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name=primary_metric_name,
                                     primary_metric_goal=primary_metric_goal,
                                     policy=policy,
                                     max_total_runs=20,
                                     max_concurrent_runs=3,
                                     )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperopt_run = exp.submit(hyperdrive_config, tag={'type':'hyperdrive_opt'})
RunDetails(hyperopt_run).show()
### YOUR CODE HERE ###

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
# Get your best run and save the model from that run.
hyperopt_run.wait_for_completion(show_output=True)
assert (hyperopt_run.get_status() == "Completed")
best_run = hyperopt_run.get_best_run_by_primary_metric()

best_run.register_model(model_name="bankmarketing_hyperdrive", model_path="outputs/bankmarketing_hyperdrive.joblib")

RunId: HD_81d8b077-4f2d-4d85-a760-4f6b16ab57e7
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_81d8b077-4f2d-4d85-a760-4f6b16ab57e7?wsid=/subscriptions/572f8abf-a1a0-4b78-8c6d-3630739c72b5/resourcegroups/aml-quickstarts-130459/workspaces/quick-starts-ws-130459

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-14T09:10:26.743506][API][INFO]Experiment created<END>\n""<START>[2020-12-14T09:10:27.382144][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n""<START>[2020-12-14T09:10:27.737980][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-14T09:10:29.3465522Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_81d8b077-4f2d-4d85-a760-4f6b16ab57e7
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_81d8b077-4f2d-4d85-a760-4f6b16ab57e7?wsid=/subscriptions/572f8

Model(workspace=Workspace.create(name='quick-starts-ws-130459', subscription_id='572f8abf-a1a0-4b78-8c6d-3630739c72b5', resource_group='aml-quickstarts-130459'), name=bankmarketing_hyperdrive, id=bankmarketing_hyperdrive:1, version=1, tags={}, properties={})

In [18]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
bankmarketing_data = TabularDatasetFactory.from_delimited_files(url)

In [19]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(bankmarketing_data)

In [20]:
from azureml.train.automl import AutoMLConfig


# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
x["y"] = y # add target to training data
ds = ws.get_default_datastore()
input_data = TabularDatasetFactory.register_pandas_dataframe(dataframe=x, target=ds, name="bankmarketing")

automl_settings = {
    "n_cross_validations": 3,
    "enable_early_stopping": True,
    "primary_metric": "accuracy",
    "blocked_models": ["SVM"], # blocked due to long execution time compared to other models
    "enable_onnx_compatible_models": True
}
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    compute_target=aml_compute,
    training_data=input_data,
    label_column_name="y",
    **automl_settings)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/31ec5d62-1135-4440-8750-1c8ee3173726/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [21]:
# Submit your automl run
automl_run = exp.submit(automl_config, tag={'type':'automl'}, show_output=True)


Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_348f3894-6b74-4a93-9751-9e6e4b82cd14

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards o

In [28]:
# Retrieve and save your best automl model.
from azureml.automl.runtime.onnx_convert import OnnxConverter
automl_run.wait_for_completion()
assert (automl_run.get_status() == "Completed")
best_ml_run, onnx_model = automl_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, file_path="best_automl.onnx")
